import modules & 

In [14]:
import pandas as pd
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: d:\myhub\Project1\mid_code


load data

In [15]:
jpy = pd.read_csv('../dataset/JPY_KRW.csv')[['날짜','종가','시가']].rename(
    columns={'날짜':'date',
             '종가':'close',
             '시가':'open'})

In [16]:
jpy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    66 non-null     object 
 1   close   66 non-null     float64
 2   open    66 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.7+ KB


#### 환율 변동값 전처리
* 전일 변동율 = (전날종가-전날시가)/전날시가 yesterday_change <br>
* 야간 변동율 = (당일시가-전날종가)/전날종가 night_change <br>
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [17]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
jpy['jpy_yesterday_change'] = round((jpy['close']-jpy['open'])/jpy['open']*100, 2)
jpy['jpy_yesterday_change'] = jpy['jpy_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
jpy['close_shift'] = jpy['close'].shift(1)
jpy['jpy_night_change'] = round((jpy['open']-jpy['close_shift'])/jpy['close_shift']*100,2)

jpy.drop('close_shift',axis=1,inplace=True)

In [18]:
# 데이터 기간 : 2023/07/31 ~ 2023/10/20
# jpy = jpy.loc[jpy['date']<='2023-10-20',].loc[jpy['date']>='2023-07-31',]
jpy = jpy[['date','jpy_yesterday_change','jpy_night_change']]

In [19]:
# 날짜순으로 오름차순 정렬
jpy.sort_values(by='date',inplace=True)
jpy.reset_index(drop=True, inplace=True)

In [20]:
jpy

,date,jpy_yesterday_change,jpy_night_change
0,2023- 07- 28,-0.56,2.55
1,2023- 07- 31,0.10,0.42
2,2023- 08- 01,0.74,-0.87
3,2023- 08- 02,0.49,-1.27
4,2023- 08- 03,0.86,-1.43
...,...,...,...
61,2023- 10- 23,-0.11,0.57
62,2023- 10- 24,0.37,-0.30
63,2023- 10- 25,-0.15,-0.29
64,2023- 10- 26,0.25,-0.03


In [21]:
jpy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  66 non-null     object 
 1   jpy_yesterday_change  65 non-null     float64
 2   jpy_night_change      65 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.7+ KB


기간 통일

In [22]:
ks200 = pd.read_csv('../dataset/clean_feature/ks200_present.csv', index_col=0)
ks200.date = pd.to_datetime(ks200.date)
# 데이터 타입 변경
jpy['date'] = pd.to_datetime(jpy['date'])
jpy = pd.merge(jpy, ks200, left_on='date', right_on='date', how='inner')[jpy.columns]

In [23]:
print(jpy.head(), jpy.tail())

        date  jpy_yesterday_change  jpy_night_change
0 2023-07-31                  0.10              0.42
1 2023-08-01                  0.74             -0.87
2 2023-08-02                  0.49             -1.27
3 2023-08-03                  0.86             -1.43
4 2023-08-04                 -0.30             -0.57          date  jpy_yesterday_change  jpy_night_change
49 2023-10-16                  0.13              0.24
50 2023-10-17                 -0.06             -0.13
51 2023-10-18                  0.15             -0.20
52 2023-10-19                 -0.65              0.42
53 2023-10-20                 -0.57              1.08


In [24]:
jpy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  54 non-null     datetime64[ns]
 1   jpy_yesterday_change  54 non-null     float64       
 2   jpy_night_change      54 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.4 KB


csv 파일로 새로 저장

In [25]:
jpy.to_csv("../dataset/clean_feature/jpy_krw_final.csv")